>### VGG

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

#### Data

In [91]:
#根据不同模型的input要求处理图片
def preprocess(image, label): 
    image = tf.cast(image, tf.float32) 
    image = keras.applications.vgg16.propecess_input() #比如用mean和std来normalize使得mean=0，把RGB转成BGR等
    return image, label

#### Model
Paper: [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

<img src='https://miro.medium.com/max/1000/1*_vGloND6yyxFeFH5UyCDVg.png'></img>

In [171]:
class Conv(keras.Model):
    def __init__(self, filters, kernel_size=3, activation='relu'):
        super(Conv, self).__init__()
        self.conv = layers.Conv2D(filters, kernel_size, activation=activation, padding= 'same',
                                  kernel_regularizer=regularizers.l2(l2=5e-5))
    def call(self, inputs):
        x = self.conv(inputs)
        return x

In [277]:
class VGG(keras.Model):
    def __init__(self, name, n_convs=None, n_classes=1000):
        super(VGG, self).__init__(name=name)
        if not n_convs:
            versions = {'vgg16':[2,2,3,3,3],
                        'vgg19':[2,2,4,4,4]}
            n_convs = versions[name]
        
        self.block_1 = self.build_blocks(n_convs[0], 64, 1)
        self.block_2 = self.build_blocks(n_convs[1], 128, 2)
        self.block_3 = self.build_blocks(n_convs[2], 256, 3)
        self.block_4 = self.build_blocks(n_convs[3], 512, 4)
        self.block_5 = self.build_blocks(n_convs[4], 512, 5)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(4096, activation='relu', name='fc1')
        self.fc2 = layers.Dense(4096, activation='relu', name='fc2')
        self.classifier = layers.Dense(n_classes, activation='softmax', name='predictions')

    def build_blocks(self, n_conv, filters, block, pool_size=2, pool_stride=2):
        block = keras.Sequential([],name=f'block{block}')
        for i in range(n_conv):
            block.add(Conv(filters))
        block.add(layers.MaxPool2D(pool_size, strides=pool_stride))
        return block
    
    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.block_5(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.classifier(x)
        return x

In [278]:
# 本来想用vars(self)[f'conv{i}']来循环创建layers，但是不知道为什么layers会没有参数
inputs = keras.Input(shape=(224,224,3))
model = build_vgg('vgg16')
model.build(input_shape=(None,224,224,3))
model.call(inputs)
model.summary()

<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'predictions')>

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1 (Sequential)          (None, 112, 112, 64)      38720     
_________________________________________________________________
block2 (Sequential)          (None, 56, 56, 128)       221440    
_________________________________________________________________
block3 (Sequential)          (None, 28, 28, 256)       1475328   
_________________________________________________________________
block4 (Sequential)          (None, 14, 14, 512)       5899776   
_________________________________________________________________
block5 (Sequential)          (None, 7, 7, 512)         7079424   
_________________________________________________________________
flatten_20 (Flatten)         (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              102764

In [ ]:
#初始学习率为0.001，当验证集准确率不再提高时，学习率衰减为原来的0.1倍，总共下降三次
lr_scheduler = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-3 * 1/(1+epoch))
model.compile(optimizer=keras.optimizers.SGD(momentum=0.9))
model.fit(X,y, batch_size=256, epochs=74)

#### Pretrained

In [166]:
from tensorflow.keras.applications import VGG16, VGG19

In [167]:
model = VGG16(input_shape=(224,224,3), include_top=True,
              weights=None, classes=1000) 
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     